In [ ]:
import os
import numpy as np
import time
import datetime
import statistics

divider = "=" * 75

def startFeature():
    print("Welcome to the Student Management System")

    student_level = input("Please select your student level (U for Undergraduate, G for Graduate, B for Both): ").upper()
    while student_level not in ['U', 'G', 'B']:
        print("Invalid input. Please choose from U, G, or B.")
        student_level = input("Please select your student level (U for Undergraduate, G for Graduate, B for Both): ").upper()

    graduate_level = None
    if student_level in ['G', 'B']:
        graduate_level = input("Please select your graduate level (M for Master, D for Doctorate, B0 for Both): ").upper()
        while graduate_level not in ['M', 'D', 'B0']:
            print("Invalid input. Please choose from M, D, or B0.")
            graduate_level = input("Please select your graduate level (M for Master, D for Doctorate, B0 for Both): ").upper()

    student_id = input("Please enter your 9-digit student ID (e.g. 202006000): ")
    while not student_id.isdigit() or len(student_id) != 9:
        print("Invalid student ID. Please ensure it is exactly 9 digits.")
        student_id = input("Please enter your 9-digit student ID (e.g. 202006000): ")
        
    print("Processing your selection...")
    time.sleep(3)
    menuFeature(student_level, graduate_level, student_id)

    
def menuFeature(student_level, graduate_level, student_id):
    session_requests = []  # Keep track of requests made during the session

    while True:
        print("\nStudent Management System Menu")
        print("1. Student Details")
        print("2. Statistics")
        print("3. Transcript based on major courses")
        print("4. Transcript based on minor courses")
        print("5. Full transcript")
        print("6. Previous transcript requests")
        print("7. Select another student")
        print("8. Terminate the system")

        choice = input("Please select an option (1-8): ")

        if choice == '1':
            processRequest("Student Details")
            studentDetails(student_id)
            session_requests.append("Student Details")
        elif choice == '2':
            processRequest("Statistics")
            statisticsFeature(student_level, graduate_level, student_id)
            session_requests.append("Statistics")
        elif choice == '3':
            processRequest("Major Transcript")
            majorTranscriptFeature(student_id, "Major")
            session_requests.append("Major Transcript")
        elif choice == '4':
            processRequest("Minor Transcript")
            minorTranscriptFeature(student_id, "Minor")
            session_requests.append("Minor Transcript")
        elif choice == '5':
            processRequest("Full Transcript")
            fullTranscriptFeature(student_id, "Full")
            session_requests.append("Full Transcript")
        elif choice == '6':
            previousRequests(student_id, session_requests)
        elif choice == '7':
            time.sleep(3.5)  # Delay before starting a new feature
            startFeature()
            return  # Exit the menu and restart the program
        elif choice == '8':
            terminateSystem(student_id, session_requests)
            break
        else:
            print("Invalid choice. Please try again.")

def processRequest(request_name):
    print(f"Processing your request for {request_name}...")
    time.sleep(3.5)  # Delay before showing the result

def studentDetails(student_id):
    folder_path = r"C:\\Users\\admin\\JupyterJade"
    file_name = "studentDetails.csv"
    file_path = os.path.join(folder_path, file_name)

    if not os.path.exists(file_path):
        print(f"No file found for Student ID {student_id}.")
        return

    try:
        data = np.loadtxt(file_path, delimiter=',', dtype=str, skiprows=1)  # Use numpy.loadtxt here
        student_data = None
        for row in data:
            if row[1] == student_id:  # Assuming stdID is in the second column
                student_data = row
                break

        if student_data is None:
            print(f"No student found with ID {student_id}.")
        else:
            print("\nStudent Details:")
            print("-" * 30)
            print(f"{'Student ID:':<20} {student_data[1]}")
            print(f"{'Name:':<20} {student_data[2]}")
            print(f"{'College:':<20} {student_data[3]}")
            print(f"{'Department:':<20} {student_data[4]}")
            print(f"{'Level:':<20} {student_data[5]}")
            print(f"{'Degree:':<20} {student_data[6]}")
            print(f"{'Major:':<20} {student_data[7]}")
            print(f"{'Minor:':<20} {student_data[8]}")
            print(f"{'Terms:':<20} {student_data[9]}")
        logPreviousRequest(student_id, "Student Details")
    except Exception as e:
        print(f"Error loading file: {e}")
    returnToMenu()

def details(student_id):
    folder_path = r"C:\\Users\\admin\\JupyterJade"
    file_name = "studentDetails.csv" 
    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'r') as file:
        data = np.loadtxt(file, delimiter=',', dtype=str, skiprows=1)
    
    student_found = False
    for line in data:
        if student_id == line[1]:  # Assuming the student ID is in column 1
            print(f"Found student {student_id}!")
            print(
                f" \nName: {line[2]:<35}stdID: {line[1]}\n",
                f"College: {line[4]:<32}Department: {line[3]}\n",
                f"Major: {line[7]:<34}Minor: {line[8]}\n",
                f"level: {line[5]:<34}Number of terms: {line[9]}",)
            student_found = True
            break

    if not student_found:
        print(f"No details found for Student ID {student_id}")

def statisticsFeature(student_level, graduate_level, student_id):
    try:
        folder_path = r"C:\\Users\\admin\\JupyterJade"
        file_name = f"C:\\Users\\admin\\JupyterJade\\{student_id}.csv"
        file_path = os.path.join(folder_path, file_name)
        
        # Load the data, skipping the header row
        data = np.loadtxt(file_name, delimiter=",", dtype=str, skiprows=1)

        # Extract relevant columns: level, scores, and course type
        levels = data[:, 0]  # Assuming 1st column contains the level (G/U)
        scores = data[:, 7].astype(float)
        terms = data[:, 2]  # Assuming 3rd column contains term numbers
        level_names = {'G': 'Graduate Level', 'U': 'Undergraduate Level'}

        # Initialize a dictionary to store statistics by level
        statistics = {}

        for level_type in ['G', 'U']:  # Graduate and Undergraduate
            # Filter data for the current level
            level_indices = levels == level_type
            level_scores = scores[level_indices]
            level_terms = terms[level_indices]

            if level_scores.size > 0:
                # Filter rows based on term numbers for this level
                term1_scores = level_scores[level_terms == "1"]
                term2_scores = level_scores[level_terms == "2"]

                # Calculate averages for the current level
                overall_average = np.mean(level_scores)
                term1_average = np.mean(term1_scores) if term1_scores.size > 0 else 0.0
                term2_average = np.mean(term2_scores) if term2_scores.size > 0 else 0.0

                # Determine maximum and minimum scores and their corresponding terms
                max_score = np.max(level_scores)
                min_score = np.min(level_scores)

                max_term = "Term 1" if max_score in term1_scores else "Term 2"
                min_term = "Term 1" if min_score in term1_scores else "Term 2"

                # Store statistics in the dictionary
                statistics[level_type] = {
                    "Overall Average": overall_average,
                    "Term 1": term1_average,
                    "Term 2": term2_average,
                    "Max Score": max_score,
                    "Max Term": max_term,
                    "Min Score": min_score,
                    "Min Term": min_term,
                }

        # Save statistics to a file
        output_file_name = f"C:\\Users\\admin\\JupyterJade\\{student_id}statistics.txt"
        with open(output_file_name, "w") as file:
            for level_type, stats in statistics.items():
                level_name = level_names.get(level_type, level_type)  # Get the full name or use the type as fallback
                file.write("\n==========================================================\n")
                file.write(f"\n***************    {level_name}    ***************")
                file.write("\n==========================================================\n")
                file.write(f"\nOverall Average (major and minor) for all terms: {stats['Overall Average']:.2f}")
                file.write("\nAverage (major and minor) of each term:")
                file.write(f"\n      Term 1: {stats['Term 1']:.2f}")
                file.write(f"\n      Term 2: {stats['Term 2']:.2f}")
                file.write(f"\nMaximum grade(s) and in which terms(s): {stats['Max Score']:.2f} ({stats['Max Term']})")
                file.write(f"\nMinimum grade(s) and in which terms(s): {stats['Min Score']:.2f} ({stats['Min Term']})")
                file.write("\n")
        # Print statistics to the console
        for level_type, stats in statistics.items():
            level_name = level_names.get(level_type, level_type)  # Get the full name or use the type as fallback
            print("\n==========================================================\n")
            print(f"***************    {level_name}    ***************")
            print("\n==========================================================\n")
            print(f"Overall Average (major and minor) for all terms: {stats['Overall Average']:.2f}")
            print("Average (major and minor) of each term:")
            print(f"      Term 1: {stats['Term 1']:.2f}")
            print(f"      Term 2: {stats['Term 2']:.2f}")
            print(f"Maximum grade(s) and in which terms(s): {stats['Max Score']:.2f} ({stats['Max Term']})")
            print(f"Minimum grade(s) and in which terms(s): {stats['Min Score']:.2f} ({stats['Min Term']})")
            print("\n==========================================================\n")

        print("\nStatistics saved to file.")
        time.sleep(2)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        menuFeature(student_level, graduate_level, student_id)

def majorTranscriptFeature(student_id, major_or_minor):
    details(student_id)  # Pass student_id to the details function
    
    # Prepare the header content for the Major Transcript
    transcript_content = (
        f"{divider}\n***************************      Major Transcript      *****************************\n{divider}\n"
        f"{'course ID':<12} {'course Name':<30} {'credit hours':<20} {'grade':<10}\n"
    )
    
    try:
        # Load the student data from the CSV file
        data = np.loadtxt(f'{student_id}.csv', delimiter=',', dtype=str, skiprows=1)
        
        total = 0
        count = 0
        
        # Loop through Term 1 data and append the relevant data to the transcript content
        for line in data:
            if line[5] == 'Major' and line[2] == '1':  # Ensure Term 1 data and 'Major' filter
                transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                total += float(line[7])
                count += 1
        
        # Calculate the average for Term 1
        average = total / count if count else 0

        # Calculate overall average for all courses in Term 1
        overtotal = sum(float(line[7]) for line in data if line[2] == '1')
        overcount = sum(1 for line in data if line[2] == '1')
        overaverage = overtotal / overcount if overcount else 0
        overaverage = round(overaverage, 2)
        
        transcript_content += f"\n{'Major'} Average = {average:<15}\t\tOverall Average = {overaverage}\n"
        
    except Exception as e:
        print(f"Error loading Term 1 data: {e}")

    # Term 2 section
    transcript_content += (
        f"{divider}\n***************************      Term 2      *****************************\n{divider}\n"
        f"{'course ID':<12} {'course Name':<30} {'credit hours':<20} {'grade':<10}\n"
    )

    try:
        total = 0
        count = 0
        
        # Loop through Term 2 data and append the relevant data to the transcript content
        for line in data:
            if line[5] == 'Major' and line[2] == '2':  # Ensure Term 2 data and 'Major' filter
                transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                total += float(line[7])
                count += 1
        
        # Calculate the average for Term 2
        average = total / count if count else 0

        # Calculate overall average for all courses in Term 2
        overtotal = sum(float(line[7]) for line in data if line[2] == '2')
        overcount = sum(1 for line in data if line[2] == '2')
        overaverage = overtotal / overcount if overcount else 0
        overaverage = round(overaverage, 2)
        
        transcript_content += f"\n{'Major'} Average = {average:<15}\t\tOverall Average = {overaverage}\n"
        
        # Save the Major Transcript content to a file
        saveTranscriptToFile(student_id, transcript_content, major_or_minor="Major")
    
    except Exception as e:
        print(f"Error loading Term 2 data: {e}")

    # Final footer and logging
    transcript_content += f"{divider}\n********************      End of Transcript      **************************\n{divider}\n"
    logPreviousRequest(student_id, f"{'Major'} Transcript")
    
    # Print the transcript content to display
    print(transcript_content)

def minorTranscriptFeature(student_id, major_or_minor):
    details(student_id)  # Pass student_id to the details function
    
    # Initialize the content of the transcript
    transcript_content = (
        f"{divider}\n***************************      Minor Transcript      *****************************\n{divider}\n"
        f"{'course ID':<12} {'course Name':<30} {'credit hours':<20} {'grade':<10}\n"
    )
    
    try:
        data = np.loadtxt(f'{student_id}.csv', delimiter=',', dtype=str, skiprows=1)  # Load student data from CSV
        total = 0
        count = 0
        
        # Process Term 1 data for Minor courses
        for line in data:
            if line[5] == 'Minor' and line[2] == '1':  # Ensure it's a Minor course in Term 1
                transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                total += float(line[7])
                count += 1
        
        # Calculate average for Term 1
        average = total / count if count else 0

        # Calculate overall average for Term 1
        overtotal = sum(float(line[7]) for line in data if line[2] == '1')
        overcount = sum(1 for line in data if line[2] == '1')
        overaverage = overtotal / overcount if overcount else 0
        overaverage = round(overaverage, 2)

        transcript_content += f"\n{'Minor'} Average = {average:<15}\t\tOverall Average = {overaverage}\n"
    
    except Exception as e:
        print(f"Error loading Term 1 data: {e}")

    # Process Term 2 data for Minor courses
    transcript_content += (
        f"{divider}\n***************************      Term 2      *****************************\n{divider}\n"
        f"{'course ID':<12} {'course Name':<30} {'credit hours':<20} {'grade':<10}\n"
    )

    try:
        total = 0
        count = 0
        
        # Loop through Term 2 data for Minor courses
        for line in data:
            if line[5] == 'Minor' and line[2] == '2':  # Ensure it's a Minor course in Term 2
                transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                total += float(line[7])
                count += 1
        
        # Calculate average for Term 2
        average = total / count if count else 0

        # Calculate overall average for Term 2
        overtotal = sum(float(line[7]) for line in data if line[2] == '2')
        overcount = sum(1 for line in data if line[2] == '2')
        overaverage = overtotal / overcount if overcount else 0
        overaverage = round(overaverage, 2)

        transcript_content += f"\n{'Minor'} Average = {average:<15}\t\tOverall Average = {overaverage}\n"
        
        # Save the Minor Transcript to a file
        saveTranscriptToFile(student_id, transcript_content, major_or_minor="Minor")
    
    except Exception as e:
        print(f"Error loading Term 2 data: {e}")

    # Final footer and logging
    transcript_content += f"{divider}\n********************      End of Transcript      **************************\n{divider}\n"
    logPreviousRequest(student_id, f"{'Minor'} Transcript")
    
    # Print the transcript content to display
    print(transcript_content)


def fullTranscriptFeature(student_id, major_or_minor):
    details(student_id)  # Pass student_id to the details function
    
    # Prepare content for Term 1
    transcript_content = (
        f"{divider}\n***************************      Term 1      *****************************\n{divider}\n"
        f"{'course ID':<12} {'course Name':<30} {'credit hours':<20} {'grade':<10}\n"
    )
    
    try:
        data = np.loadtxt(f'{student_id}.csv', delimiter=',', dtype=str, skiprows=1)  # Load student data from CSV
        total_major = 0
        total_minor = 0
        count_major = 0
        count_minor = 0
        
        # Loop through data to collect Major and Minor subjects for Term 1
        for line in data:
            if line[2] == '1':  # Term 1
                if line[5] == 'Major':
                    transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                    total_major += float(line[7])
                    count_major += 1
                elif line[5] == 'Minor':
                    transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                    total_minor += float(line[7])
                    count_minor += 1
        
        # Calculate averages for Major and Minor for Term 1
        major_average = total_major / count_major if count_major else 0
        minor_average = total_minor / count_minor if count_minor else 0
        
        # Calculate overall average for Term 1
        term1_total = total_major + total_minor
        term1_count = count_major + count_minor
        overall_average_term1 = term1_total / term1_count if term1_count else 0

        transcript_content += (
            f"Major Average = {major_average:<15}\t\tMinor Average = {minor_average:<15}\n"
            f"Overall Average (Term 1) = {round(overall_average_term1, 2)}\n"
        )
    except Exception as e:
        print(f"Error loading Term 1 data: {e}")
    
    # Prepare content for Term 2
    transcript_content += (
        f"{divider}\n***************************      Term 2      *****************************\n{divider}\n"
        f"{'course ID':<12} {'course Name':<30} {'credit hours':<20} {'grade':<10}\n"
    )
    
    try:
        total_major = 0
        total_minor = 0
        count_major = 0
        count_minor = 0
        
        # Loop through data to collect Major and Minor subjects for Term 2
        for line in data:
            if line[2] == '2':  # Term 2
                if line[5] == 'Major':
                    transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                    total_major += float(line[7])
                    count_major += 1
                elif line[5] == 'Minor':
                    transcript_content += f"{line[4]:<12}{line[3]:<40}{line[6]:<15}{line[7]:<10}\n"
                    total_minor += float(line[7])
                    count_minor += 1
        
        # Calculate averages for Major and Minor for Term 2
        major_average = total_major / count_major if count_major else 0
        minor_average = total_minor / count_minor if count_minor else 0
        minor_average = round(minor_average, 2)
        
        # Calculate overall average for Term 2
        term2_total = total_major + total_minor
        term2_count = count_major + count_minor
        overall_average_term2 = term2_total / term2_count if term2_count else 0
        
        # Calculate the overall average for both terms (combined)
        total_combined = term1_total + term2_total
        count_combined = term1_count + term2_count
        overall_average = total_combined / count_combined if count_combined else 0
        overall = round(overall_average, 2)

        transcript_content += (
            f"Major Average = {major_average:<15}\t\tMinor Average = {minor_average:<15}\n"
            f"Overall Average (Term 2) = {round(overall_average_term2, 2)}\n"
            f"OVERALL AVERAGE = {overall}\n"
        )
        
    except Exception as e:
        print(f"Error loading Term 2 data: {e}")

    # End of the transcript content
    transcript_content += f"\n{divider}\n********************      End of Transcript      **************************\n{divider}\n"
    
    # Log the previous request
    logPreviousRequest(student_id, f"{'Major and Minor'} Transcript")

    # Print the full transcript content
    print(transcript_content)


def previousRequests(student_id, session_requests):
    print("\nPrevious Requests:")
    print("-" * 30)
    if not session_requests:
        print("No requests made during this session.")
    else:
        print(f"{'Request':<30} {'Date and Time'}")
        print("-" * 30)
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        for request in session_requests:
            print(f"{request:<30} {current_time}")
    returnToMenu()

def saveTranscriptToFile(student_id, transcript_content, major_or_minor=None):
    # Determine the filename based on the type of transcript (Major, Minor, or Full)
    if major_or_minor == 'Major':
        filename = f"std{student_id}MajorTranscript.txt"
    elif major_or_minor == 'Minor':
        filename = f"std{student_id}MinorTranscript.txt"
    else:
        filename = f"std{student_id}FullTranscript.txt"
    
    try:
        # Write the transcript content to the file
        with open(filename, 'w') as file:
            file.write(transcript_content)
        print(f"Transcript saved successfully as {filename}")
    except Exception as e:
        print(f"Error saving transcript: {e}")

def logPreviousRequest(student_id, request_name):
    folder_path = r"C:\\Users\\admin\\JupyterJade"
    file_path = os.path.join(folder_path, f"std{student_id}PreviousRequests.txt")
    with open(file_path, 'a') as file:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        file.write(f"{request_name} - {current_time}\n")
    
def terminateSystem(student_id, session_requests):
    print("\nTerminating the system...")
    logPreviousRequest(student_id, "System Termination")1
    
    time.sleep(3)
    print("System has been terminated.")

def returnToMenu():
    print("\nReturning to menu...")
    time.sleep(2.5)  # Delay before returning to the menu

# Start the program
startFeature()


Welcome to the Student Management System


Please select your student level (U for Undergraduate, G for Graduate, B for Both):  g
Please select your graduate level (M for Master, D for Doctorate, B0 for Both):  m
Please enter your 9-digit student ID (e.g. 202006000):  202415252


Processing your selection...

Student Management System Menu
1. Student Details
2. Statistics
3. Transcript based on major courses
4. Transcript based on minor courses
5. Full transcript
6. Previous transcript requests
7. Select another student
8. Terminate the system


Please select an option (1-8):  2


Processing your request for Statistics...


***************    Graduate Level    ***************


Overall Average (major and minor) for all terms: 90.60
Average (major and minor) of each term:
      Term 1: 90.70
      Term 2: 90.50
Maximum grade(s) and in which terms(s): 99.00 (Term 1)
Minimum grade(s) and in which terms(s): 83.00 (Term 1)




***************    Undergraduate Level    ***************


Overall Average (major and minor) for all terms: 91.80
Average (major and minor) of each term:
      Term 1: 93.80
      Term 2: 89.80
Maximum grade(s) and in which terms(s): 97.00 (Term 1)
Minimum grade(s) and in which terms(s): 86.00 (Term 2)



Statistics saved to file.

Student Management System Menu
1. Student Details
2. Statistics
3. Transcript based on major courses
4. Transcript based on minor courses
5. Full transcript
6. Previous transcript requests
7. Select another student
8. Terminate the system


...

Processing your request for Minor Transcript...
Found student 202415252!
 
Name: Mary Jade Galban                   stdID: 202415252
 College: Computer Engineering            Department: Engineering
 Major: BS1Bigdata                        Minor: BS1MachineLearning
 level: G                                 Number of terms: 2
Transcript saved successfully as std202415252MinorTranscript.txt
***************************      Minor Transcript      *****************************
course ID    course Name                    credit hours         grade     
CWTS 001    Civil Welfare Trainning Ser 1           2              83        
GEED 004    Math in the Modern World                2              86        
GEED 007    Scienc Technology and Society           2              93        
MATH 004    Advanced Math                           2              83        
CMPE 104    Computer Programming Management         2              98        
DABD 001    Data Analytics and Big Data Systems     2   

Please select an option (1-8):  6



Previous Requests:
------------------------------
Request                        Date and Time
------------------------------
Student Details                2025-02-04 00:03:20
Major Transcript               2025-02-04 00:03:20
Minor Transcript               2025-02-04 00:03:20

Returning to menu...

Student Management System Menu
1. Student Details
2. Statistics
3. Transcript based on major courses
4. Transcript based on minor courses
5. Full transcript
6. Previous transcript requests
7. Select another student
8. Terminate the system


Please select an option (1-8):  8



Terminating the system...
System has been terminated.

Student Management System Menu
1. Student Details
2. Statistics
3. Transcript based on major courses
4. Transcript based on minor courses
5. Full transcript
6. Previous transcript requests
7. Select another student
8. Terminate the system
